# Part 1: Creating your first Thing using AWS IoT


To start the workshop we are going to create a Thing inside AWS IoT. A Thing should represent any phisycal thing in world: button, a raspberry pi, a lamp or even your own laptop can be a thing. We are going to identify this thing by a name starting with MyHealthThing followed by six numbers.

In [23]:
%store -r home
home

'/home/ec2-user'

Create a Thing, a unique name helps to make references more clear.
Any computer can be a thing, lets use this notebook instance as our thing to begin.

In [24]:
%store -r HM
HM

'110602'

In [25]:
thing_name = "MyHealthThing{}".format(HM)
thing_name

'MyHealthThing110602'

In [26]:
! aws iot create-thing --thing-name {thing_name}

{
    "thingArn": "arn:aws:iot:us-east-1:977842192436:thing/MyHealthThing110602", 
    "thingName": "MyHealthThing110602", 
    "thingId": "2d8d136e-6a92-4af0-9c0d-988206e6209f"
}


# Authentication and Authorization


Create a certificate and keys to authenticate your thing. 

In [27]:
! mkdir -p "{home}/aws_iot"

certificate_file = "{}/aws_iot/certificate{}.pem".format(home,HM)
public_key = "{}/aws_iot/public-key{}.pem".format(home,HM)
private_key = "{}/aws_iot/private-key{}.pem".format(home,HM)

%store certificate_file
%store public_key
%store private_key

Stored 'certificate_file' (str)
Stored 'public_key' (str)
Stored 'private_key' (str)


In [28]:
certificateArn = ! aws iot create-keys-and-certificate \
  --set-as-active \
  --certificate-pem-outfile {certificate_file} \
  --public-key-outfile {public_key} \
  --private-key-outfile {private_key} \
  --query certificateArn \
  --output text
certificate_arn = certificateArn.s
certificate_arn

'arn:aws:iot:us-east-1:977842192436:cert/57d0c9073ae3c74634726be6e15c7efed72f6082aa36ff24b2d46b8ee423d964'

Download the root Certification Authority used by AWS IoT


In [29]:
ca_file = "{}/aws_iot/ca{}.pem".format(home,HM)
%store ca_file
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {ca_file}

Stored 'ca_file' (str)
--2018-05-22 13:26:38--  https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem
Resolving www.symantec.com (www.symantec.com)... 104.112.44.114, 2600:1408:20:397::145b, 2600:1408:20:398::145b
Connecting to www.symantec.com (www.symantec.com)|104.112.44.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1758 (1.7K) [text/plain]
Saving to: ‘/home/ec2-user/aws_iot/ca110602.pem’

/home/ec2-user/aws_ 100%[===================>]   1.72K  --.-KB/s    in 0s      

2018-05-22 13:26:38 (324 MB/s) - ‘/home/ec2-user/aws_iot/ca110602.pem’ saved [1758/1758]



Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [30]:
! printf "Private Key {private_key} " && [ -f {private_key} ] && printf "found" || printf "not found"
! printf "\nPublic Key {public_key} " && [ -f {public_key} ] && printf "found" || printf "not found"
! printf "\nCertificate {certificate_file} " && [ -f {certificate_file} ] && printf "found" || printf "not found"
! printf "\nCertification Authority {ca_file} " && [ -f {ca_file} ] && printf "found" || printf "not found"

Private Key /home/ec2-user/aws_iot/private-key110602.pem found
Public Key /home/ec2-user/aws_iot/public-key110602.pem found
Certificate /home/ec2-user/aws_iot/certificate110602.pem found
Certification Authority /home/ec2-user/aws_iot/ca110602.pem found

Create an IAM authorization policy stating what is going to be allowed.

In [31]:
#TODO: Use a more restrictive policy
policy_document = '''{
  "Version": "2012-10-17",
  "Statement": [{
    "Effect": "Allow",
    "Action": "iot:*",
    "Resource": "*"}]
}'''
policy_document

'{\n  "Version": "2012-10-17",\n  "Statement": [{\n    "Effect": "Allow",\n    "Action": "iot:*",\n    "Resource": "*"}]\n}'

In [32]:
policy_name = "NotebooksPolicy{}".format(HM)
policyArn = ! aws iot create-policy \
    --policy-name {policy_name} \
    --policy-document '{policy_document}' \
    --query policyArn \
    --output text
policyArn.s

' An error occurred (ResourceAlreadyExistsException) when calling the CreatePolicy operation: Policy cannot be created - name already exists (name=NotebooksPolicy110602)'

Attach certificate to thing and policy

In [33]:
! aws iot attach-policy \
    --policy-name {policy_name} \
    --target {certificate_arn}

In [34]:
! aws iot attach-thing-principal \
  --thing-name {thing_name} \
  --principal {certificate_arn}

# Connecting to AWS IoT

AWS IoT provides account-specific endpoints for your things to connect:

In [35]:
endpoint_address = !aws iot describe-endpoint --query endpointAddress --output=text
endpoint_address = endpoint_address.s
%store endpoint_address
endpoint_address

Stored 'endpoint_address' (str)


'a2enzbgohiblz2.iot.us-east-1.amazonaws.com'

Ready to go! Create your MQTT client instance:

In [36]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient{}".format(HM)
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt

# Receiving and Sending Messages

Connect to the service to send and receive messages:

In [37]:
mqtt.connect()

True

Subscribe to a topic:

In [38]:
topic_name ="myhealthcare/data"
%store topic_name
subscribe_qos = 1
def custom_callback(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8") )
    print("\n--------------\n")
    
mqtt.subscribe(topic_name, subscribe_qos, custom_callback)

Stored 'topic_name' (str)


True

Publish to a topic:

In [39]:
import uuid
import json
import datetime 
import random
import time
def genPayload():
    names = ["Sarah Jane", "Rory Williams", "Amy Pond", "Jack Harkness", "Clara Oswald", "Martha Jones", "Rose Tyler"] 
    name = random.choice(names)
    id = str(uuid.uuid4())
    now = int(time.time()*1000)
    noise = round(40 * random.random());
    data = {
        "id" : id ,
        "name" : name,
        "sistolic" : 110 + noise,
        "diastolic" : 70 + noise,
        "time_stamp" : now
    }
    payload = json.dumps(data)
    return payload
payloads = []
for x in range(100):
    payloads.append(genPayload())
%store payloads
payload = random.choice(payloads)
%store payload
payload

Stored 'payloads' (list)
Stored 'payload' (str)


'{"id": "53600673-2f17-475d-9610-f25c9760f1c1", "name": "Clara Oswald", "sistolic": 127, "diastolic": 87, "time_stamp": 1526995629766}'

In [42]:
publish_qos = 0
mqtt.publish(topic_name, payload , publish_qos)

False


Received a new message: 

{"id": "53600673-2f17-475d-9610-f25c9760f1c1", "name": "Clara Oswald", "sistolic": 127, "diastolic": 87, "time_stamp": 1526995629766}

--------------


Received a new message: 

{"id": "53600673-2f17-475d-9610-f25c9760f1c1", "name": "Clara Oswald", "sistolic": 127, "diastolic": 87, "time_stamp": 1526995629766}

--------------



The subscribe_qos and publish_qos arguments refer to the quality of service parameter of the MQTT protocol. Selecting the QoS level is a tradeoff between delivery guarantee and performance, see the following resources for more infomation:

https://www.hivemq.com/blog/mqtt-essentials-part-6-mqtt-quality-of-service-levels

https://docs.aws.amazon.com//iot/latest/developerguide/protocols.html

All working fine, done for now, disconnect.

In [ ]:
mqtt.disconnect()

Now we don't want to loose this data, right?
Let's proceed to the [Storing Telemetry Data](aws-iot-storing-telemetry.ipynb) notebook right now!